In [1]:
import sys

import hvplot.pandas  # noqa
import pandas as pd

sys.path.append("..")
import subprocess

from processing import process_locust_results
from queries import generate_locust_urls

## Specify file containing results from "local" tests

In [2]:
uri = "s3://nasa-eodc-data-store/test-results/20240814010546_CMRTileTest_C2723754850-GES_DISC.json"
test_id = uri.split("/")[-1].split(".")[0]
urls_output_file = f"urls/{test_id}.csv"
kwargs = {"rescale": "0,455", "colormap_name": "coolwarm"}

## Use locust to time tile generation

In [3]:
df = generate_locust_urls(uri, urls_output_file, **kwargs)
results_output = f"results/{test_id}"
command = f"locust -f locust_titiler_cmr.py --headless --users 1 --iterations 1 --csv {results_output} --urls-file {urls_output_file} --csv-full-history --host https://dev-titiler-cmr.delta-backend.com"
subprocess.call(command, shell=True)

[2024-08-15 19:55:21,097] jupyter-maxrjones--tile-2dbenchmarking/INFO/locust.main: No run time limit set, use CTRL+C to interrupt
[2024-08-15 19:55:21,098] jupyter-maxrjones--tile-2dbenchmarking/INFO/locust.main: Starting Locust 2.30.0
Type     Name                                                                          # reqs      # fails |    Avg     Min     Max    Med |   req/s  failures/s
--------|----------------------------------------------------------------------------|-------|-------------|-------|-------|-------|-------|--------|-----------
--------|----------------------------------------------------------------------------|-------|-------------|-------|-------|-------|-------|--------|-----------
         Aggregated                                                                         0     0(0.00%) |      0       0       0      0 |    0.00        0.00

[2024-08-15 19:55:21,118] jupyter-maxrjones--tile-2dbenchmarking/INFO/locust.runners: Ramping to 1 users at a rate of 1

0

GET      /tiles/WebMercatorQuad/14/3868/12587?concept_id=C2723754850-GES_DISC&variable=precipitation&datetime=2024-02-11T00%3A00%3A00Z&backend=xarray&scale=1&return_mask=true&rescale=0%2C455&colormap_name=coolwarm     1700   1700   1700   1700   1700   1700   1700   1700   1700   1700   1700      1
GET      /tiles/WebMercatorQuad/14/5961/5957?concept_id=C2723754850-GES_DISC&variable=precipitation&datetime=2024-02-11T00%3A00%3A00Z&backend=xarray&scale=1&return_mask=true&rescale=0%2C455&colormap_name=coolwarm     1000   1000   1000   1000   1000   1000   1000   1000   1000   1000   1000      1
GET      /tiles/WebMercatorQuad/14/6098/8929?concept_id=C2723754850-GES_DISC&variable=precipitation&datetime=2024-02-11T00%3A00%3A00Z&backend=xarray&scale=1&return_mask=true&rescale=0%2C455&colormap_name=coolwarm     1500   1500   1500   1500   1500   1500   1500   1500   1500   1500   1500      1
GET      /tiles/WebMercatorQuad/14/6326/8465?concept_id=C2723754850-GES_DISC&variable=precipitation&da

## Process locust results and combine with local results

In [4]:
locust_df = process_locust_results(results_output)
df["method"] = "VEDA Hub"
df["Response Time"] = df.apply(lambda x: float(x["timings"][0]), axis=1)
df = df[["query", "Response Time", "method", "tile", "zoom"]]
combined_df = pd.concat([df, locust_df], axis=0).reset_index()

## Upload results to S3

In [5]:
results_store = f"s3://nasa-eodc-data-store/tile-benchmarking-results/{test_id}/"
subprocess.run(
    [
        "aws",
        "s3",
        "cp",
        "results/",
        results_store,
        "--recursive",
        "--exclude",
        "*",
        "--include",
        "*.csv",
    ]
)

upload: results/20240814010546_CMRTileTest_C2723754850-GES_DISC_failures.csv to s3://nasa-eodc-data-store/tile-benchmarking-results/20240814010546_CMRTileTest_C2723754850-GES_DISC/20240814010546_CMRTileTest_C2723754850-GES_DISC_failures.csv
upload: results/20240814010546_CMRTileTest_C2723754850-GES_DISC_exceptions.csv to s3://nasa-eodc-data-store/tile-benchmarking-results/20240814010546_CMRTileTest_C2723754850-GES_DISC/20240814010546_CMRTileTest_C2723754850-GES_DISC_exceptions.csv
upload: results/20240814010546_CMRTileTest_C2723754850-GES_DISC_stats.csv to s3://nasa-eodc-data-store/tile-benchmarking-results/20240814010546_CMRTileTest_C2723754850-GES_DISC/20240814010546_CMRTileTest_C2723754850-GES_DISC_stats.csv
upload: results/20240814010546_CMRTileTest_C2723754850-GES_DISC_stats_history.csv to s3://nasa-eodc-data-store/tile-benchmarking-results/20240814010546_CMRTileTest_C2723754850-GES_DISC/20240814010546_CMRTileTest_C2723754850-GES_DISC_stats_history.csv


CompletedProcess(args=['aws', 's3', 'cp', 'results/', 's3://nasa-eodc-data-store/tile-benchmarking-results/20240814010546_CMRTileTest_C2723754850-GES_DISC/', '--recursive', '--exclude', '*', '--include', '*.csv'], returncode=0)

## Plot results

In [6]:
cmap = ["#E1BE6A", "#40B0A6"]
plt_opts = {"width": 1800, "height": 400, "xrotation": 90}

In [7]:
combined_df.hvplot.box(
    y="Response Time",
    by=["zoom", "method"],
    c="method",
    cmap=cmap,
    ylabel="Response time (s)",
    xlabel="Zoom level",
    legend=False,
).opts(**plt_opts)

:BoxWhisker   [zoom,method]   (Response Time)